Apriori Algorithm

FP-Growth Algorithm

Evaluation using Support, Confidence, and Lift

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns

In [2]:
df=pd.read_csv('new_retail_data.csv')
df.head()

,Unnamed: 0,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [3]:
df.drop('Unnamed: 0', axis=1)

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302001,4246475.0,12104.0,Meagan Ellis,Courtney60@gmail.com,7.466354e+09,389 Todd Path Apt. 159,Townsville,New South Wales,4567.0,Australia,...,973.962984,Books,Penguin Books,Fiction,Bad,Same-Day,Cash,Processing,1.0,Historical fiction
302002,1197603.0,69772.0,Mathew Beck,Jennifer71@gmail.com,5.754305e+09,52809 Mark Forges,Hanover,Berlin,16852.0,Germany,...,285.137301,Electronics,Apple,Laptop,Excellent,Same-Day,Cash,Processing,5.0,LG Gram
302003,7743242.0,28449.0,Daniel Lee,Christopher100@gmail.com,9.382530e+09,407 Aaron Crossing Suite 495,Brighton,England,88038.0,UK,...,182.105285,Clothing,Adidas,Jacket,Average,Express,Cash,Shipped,2.0,Parka
302004,9301950.0,45477.0,Patrick Wilson,Rebecca65@gmail.com,9.373222e+09,3204 Baird Port,Halifax,Ontario,67608.0,Canada,...,120.834784,Home Decor,IKEA,Furniture,Good,Standard,Cash,Shipped,4.0,TV stand


Prepare the Basket Format
We’ll pivot the data using Transaction_ID and Product_Type or Product_Brand/Product_Category depending on your use case.

In [4]:
transactions = df.groupby('Transaction_ID')['products'].apply(list).values.tolist()

# Preview transaction data
print(f"Total transactions: {len(transactions)}")
print("\nSample transactions:")
for i in range(5):
    print(transactions[i])

Total transactions: 294461

Sample transactions:
['Cappuccino']
['OnePlus']
['Cooking']
['Flare jeans']
['LG G']


In [5]:
# Convert transaction data to one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
encoded_df = pd.DataFrame(te_ary, columns=te.columns_)

print("\nEncoded transaction data shape:", encoded_df.shape)
display(encoded_df.head())


Encoded transaction data shape: (294461, 318)


,4K TV,A-line dress,Acer Iconia Tab,Acer Swift,Action,Adventure,Affogato,Air conditioner,Alkaline water,Amazon Fire Tablet,...,White chocolate,Wide-leg jeans,Windbreaker,Window AC,Wireless headphones,Wrap dress,Wrench,Xiaomi Mi,iPad,iPhone
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Apriori Algorithm Implementation

In [9]:
print("Original transactions:", len(transactions))
print("Unique products:", len(te.columns_))
print("Average items per transaction:", encoded_df.sum(axis=1).mean())

Original transactions: 294461
Unique products: 318
Average items per transaction: 1.0237484760290836


In [11]:
def find_optimal_support(encoded_df, start=0.01, step=0.005):
    support = start
    while support >= 0.001:  # Don't go below 0.1%
        try:
            frequent_itemsets = apriori(encoded_df, min_support=support, use_colnames=True)
            if len(frequent_itemsets) > 0:
                print(f"Found {len(frequent_itemsets)} itemsets at support={support:.4f}")
                return support, frequent_itemsets
            support -= step
        except:
            support -= step
    raise ValueError("No frequent itemsets found even at minimum support")

optimal_support, frequent_itemsets = find_optimal_support(encoded_df)
print("\nOptimal minimum support:", optimal_support)

Found 40 itemsets at support=0.0050

Optimal minimum support: 0.005


In [13]:
# Step 1: Load and prepare data
transactions = df.groupby('Transaction_ID')['products'].apply(list).values.tolist()

# Step 2: Encode transactions with filtering
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
encoded_df = pd.DataFrame(te_ary, columns=te.columns_)

# Filter products that appear in at least N transactions
min_transactions = max(5, int(0.001 * len(transactions)))  # At least 0.1% or 5 transactions
product_counts = encoded_df.sum()
keep_products = product_counts[product_counts >= min_transactions].index
filtered_df = encoded_df[keep_products]

print(f"Keeping {len(keep_products)} products out of {len(te.columns_)}")

# Step 3: Run Apriori with dynamic support
support_values = [0.05, 0.03, 0.02, 0.01, 0.005, 0.001]  # Try from highest to lowest

for support in support_values:
    try:
        frequent_itemsets = apriori(filtered_df, min_support=support, use_colnames=True)
        if len(frequent_itemsets) > 0:
            print(f"\nSuccess with support={support:.3f}")
            print(f"Found {len(frequent_itemsets)} frequent itemsets")
            break
    except:
        continue
        
if len(frequent_itemsets) == 0:
    print("\nWarning: No frequent itemsets found with reasonable support")
else:
    # Generate association rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    print("\nTop Association Rules:")
    display(rules.sort_values('lift', ascending=False).head(10))

Keeping 317 products out of 318

Success with support=0.005
Found 40 frequent itemsets

Top Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


 FP-Growth Algorithm Implementation

In [21]:
# FP-Growth often works better with large datasets
frequent_itemsets = fpgrowth(filtered_df, min_support=0.01, use_colnames=True)
print("FP-Growth found:", len(frequent_itemsets), "itemsets")

if len(frequent_itemsets) > 0:
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    display(rules.sort_values('lift', ascending=False).head(10))

FP-Growth found: 0 itemsets


Bar chart of top frequent itemsets by support

In [22]:
# Filter rules for better visualization
strong_rules = rules_fp[(rules_fp['lift'] > 1.5) & (rules_fp['confidence'] > 0.5)]

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Support vs Confidence
axes[0, 0].scatter(strong_rules['support'], strong_rules['confidence'], alpha=0.5)
axes[0, 0].set_xlabel('Support')
axes[0, 0].set_ylabel('Confidence')
axes[0, 0].set_title('Support vs Confidence')

# Support vs Lift
axes[0, 1].scatter(strong_rules['support'], strong_rules['lift'], alpha=0.5)
axes[0, 1].set_xlabel('Support')
axes[0, 1].set_ylabel('Lift')
axes[0, 1].set_title('Support vs Lift')

# Confidence vs Lift
axes[1, 0].scatter(strong_rules['confidence'], strong_rules['lift'], alpha=0.5)
axes[1, 0].set_xlabel('Confidence')
axes[1, 0].set_ylabel('Lift')
axes[1, 0].set_title('Confidence vs Lift')

# Support vs Lift (size by confidence)
sc = axes[1, 1].scatter(strong_rules['support'], strong_rules['lift'], 
                       c=strong_rules['confidence'], cmap='viridis', alpha=0.6)
axes[1, 1].set_xlabel('Support')
axes[1, 1].set_ylabel('Lift')
axes[1, 1].set_title('Support vs Lift (Color by Confidence)')
plt.colorbar(sc, ax=axes[1, 1]).set_label('Confidence')

plt.tight_layout()
plt.show()

NameError: name 'rules_fp' is not defined

Scatter plot of rules by support, confidence and lift

In [76]:
print("\n--- ALGORITHM COMPARISON ---")
print(f"Apriori found {len(frequent_itemsets_apriori)} frequent itemsets")
print(f"FP-Growth found {len(frequent_itemsets_fpgrowth)} frequent itemsets")
print(f"Apriori generated {len(rules_apriori)} rules")
print(f"FP-Growth generated {len(rules_fpgrowth)} rules")


--- ALGORITHM COMPARISON ---
Apriori found 32 frequent itemsets
FP-Growth found 32 frequent itemsets
Apriori generated 0 rules
FP-Growth generated 0 rules
